## IBDigital Open Question Analysis
This notebook presents an analysis of answers from IBDigital Questionnaire of the following questions:  
1. Reason of flare-ups? Would it help you to know in advance that you are going to have a flare-up? Explain why (not).  
2. What should a wearable not do? DO you have any ideas what a wearable should look like?  

We use topic models to anlayze frequency of answers.

### Data Preprocessing

In [1]:
# load data
import pandas as pd
data = pd.read_excel("data/DataSet_WP2_Excel.xlsx")
# display column BB to BD, and CF, DI
df = data.iloc[:, 53:56].join(data.iloc[:, 83:84]).join(data.iloc[:, 112:113])
df["Wish_predictability_flare"] = df["Wish_predictability_flare"].map({0: "Ja", 1: "Misschien", 2: "Nee"})
# check if there are line breakers in the text of all columns
for col in df.columns:
    if df[col].astype(str).str.contains('\n').any():
        print(f"Column {col} contains line breakers.")
        df[col] = df[col].str.replace('\n', ' ', regex=True)
# add ID column
df.insert(0, "ID", range(1, 1 + len(df)))
# save to csv
#df.to_csv("data/answers.csv", index=False)
# view dataframe
df.head()

Column Reason_flare contains line breakers.
Column Unwanted_features contains line breakers.
Column View_wearable contains line breakers.


,ID,Reason_flare,Wish_predictability_flare,Wish_predictability_because,Unwanted_features,View_wearable
0,1,Nee,Ja,minder klachten vooraf,geen extra belasting,smartwatch lijkt handig vrouwen hebben niet a...
1,2,Vroeger was dat bij stress,NaN,NaN,Ingewikkeld zijn.,Niet te groot en een mooi ontwerp
2,3,Het aan- of uitstaan van het familiaire coliti...,Ja,Dan kan ik mij AANMELDEN VOOR GERICHTE BEHANDE...,NaN,NaN
3,4,NaN,NaN,NaN,Bemoedigende woorden of tips geven die ik al 1...,Door het opvallend te maken is het meteen een ...
4,5,NaN,NaN,NaN,Aanwezig op de achtergrond ipv de voorgrond. T...,NaN


### Embed text
See embed_texts.py and embed.slurm

### Load stopwords

In [1]:
# load stopwords
with open("data/stopwords-nl.txt", "r") as f:
    stopwords = [line.strip() for line in f if line.strip()] # removed "geen" from stopwords

if "geen" in stopwords:
    stopwords.remove("geen")


### Analysis 1. Unwanted Features

In [14]:
# import embeddings and documents
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
 
# define the column to analyze
TEXT_COL = "Unwanted_features"

# load embeddings and documents
emb_unwanted_features = np.load(f"embeddings/emb_robbert_{TEXT_COL}.npy")
documents_unwanted_features = pd.read_csv(f"data/answers_for_embedding_{TEXT_COL}.csv")[TEXT_COL].tolist()

# check length of documents and embeddings
assert len(emb_unwanted_features) == len(documents_unwanted_features), "Length of embeddings and documents do not match!"

# create BERTopic model for 'Unwanted_features'
print("Creating BERTopic model for 'Unwanted_features'...")
topic_model_unwanted_features = BERTopic(verbose=False)
# no dimensionality reduction is necessary, so we use BaseDimensionalityReduction which skips UMAP
topic_model_unwanted_features.umap_model = BaseDimensionalityReduction()
# Agglomerative Clustering with distance threshold
topic_model_unwanted_features.hdbscan_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.7, metric='cosine', linkage='average') 
# CountVectorizer with custom stopwords
topic_model_unwanted_features.vectorizer_model = CountVectorizer(stop_words=stopwords, min_df=2, ngram_range=(1,2))

# fit and transform
topics_unwanted_features, probs_unwanted_features = topic_model_unwanted_features.fit_transform(documents_unwanted_features, embeddings=emb_unwanted_features)
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 10)
print("Total number of 'Unwanted_features' topics: ", len(topic_model_unwanted_features.get_topic_info()))
print("Displaying top 10 topic clusters for 'Unwanted_features':")
topic_model_unwanted_features.get_topic_info().head(10)


Creating BERTopic model for 'Unwanted_features'...
Total number of 'Unwanted_features' topics:  47
Displaying top 10 topic clusters for 'Unwanted_features':


,Topic,Count,Name,Representation,Representative_Docs
0,0,345,0_meldingen_meldingen geven_teveel meldingen_teveel,"[meldingen, meldingen geven, teveel meldingen, teveel, geven meldingen, geven, stress, snel, geven teveel, onnodige]","[Stress geven, veel meldingen geven, Teveel meldingen geven, piepsignalen afgeven., Teveel meldingen geven.]"
1,1,119,1_stress_stress geven_ziekte_geven,"[stress, stress geven, ziekte, geven, geven stress, bezig, zorgen, ziekte bezig, waardoor, opvlamming]","[Stress geven, stress geven, Stress geven]"
2,2,100,2_tijd_kosten_tijd kosten_ingewikkeld,"[tijd, kosten, tijd kosten, ingewikkeld, moeilijk, teveel tijd, gebruik, ingewikkeld tijd, bedienen, ingewikkeld moeilijk]","[Te veel tijd kosten, Veel tijd kosten, Tijd kosten]"
3,3,66,3_ingewikkeld ingewikkeld_geen idee_idee_ingewikkeld geen,"[ingewikkeld ingewikkeld, geen idee, idee, ingewikkeld geen, ingewikkeld, geen, weet, gebruik ingewikkeld, ingewikkeld gebruik, ingewikkeld opladen]","[Geen idee, geen idee, geen idee]"
4,4,20,4_aanwezig_last_opvallen_onhandig,"[aanwezig, last, opvallen, onhandig, oncomfortabel, irritant, groot, lomp, dag nacht, opdringerig]","[Lelijk zijn, te groot, te aanwezig, Onhandig zijn in gebruik, te groot of opzichtig zijn, oncomfortabel zitten, Te aanwezig zijn]"
5,5,18,5_geluid_piepen_trillen_piepjes,"[geluid, piepen, trillen, piepjes, continu, eruit, geven extra, eventuele, geluidjes, piepen trillen]","[Alarmsignalen met geluid, Veel geluid maken, Geluid maken]"
6,6,18,6_informatie geven_informatie_info_verkeerde,"[informatie geven, informatie, info, verkeerde, valse, verkeerde informatie, chronische, informatie verstrekken, kwaal, geven persoonlijk]","[Verkeerde informatie geven, Verkeerde interpretabele informatie geven, Informatie geven die niet klopt.]"
7,7,14,7_data_gegevens_trekken_arts,"[data, gegevens, trekken, arts, mogelijk, opleveren, informatie, zorgen, fitbit, eenmaal]","[Onjuiste informatie verstrekken of te snel conclusies trekken: stuur data door naar mijn arts ipv zelf een conclusie te trekken, Als ik niet zelf bij alle data kan, Hij moet gewoon heel makkelijk in gebruik zijn en indien mogelijk zoveel mogelijk data verzamelen. Ik zou inderdaad niet dagelijks..."
8,8,13,8_mee_mee bezig_beperken_continu,"[mee, mee bezig, beperken, continu, bezig, teveel mee, gaat, goed gaat, stop, onzekerheid]","[Dat je er teveel mee bezig bent. Als ik er teveel mee bezig ben zorgt dat juist voor klacht en/of onzekerheid, Continu aangeven hoe het gaat zodat je er continu mee bezig bent. Dus alleen als er iets niet goed gaat., Je teveel beperken dat je er continu mee bezig moet zijn]"
9,9,13,9_delen_privacy_derden_gegevens,"[delen, privacy, derden, gegevens, data, toestemming, slecht, doorgeven, informatie, geen informatie]","[Geen Informatie delen zonder toestemming of aan derden, duidelijk over gegevens delen en privacy en dat makkelijk kunnen aanpassen., Gegevens met derden delen]"


## Explanation
We calculate the sum of all answers in all topics that contain a specific set of keywords. The count of topics containings [set of keywords] is higher than the actural occurences of the keyowrds in the original answers. This is normal and expected given that we expect the model to group together some sentences that mean similar things but don't necessarily contain these words. I think it's safe to write in the results something like:  
  
"In responses to the open-ended question 'What features do you not want for the smartwatch'", the most unwanted features were "too many notifications" or "giving stress", with around 504 (59%) responses expressing similar sentiments. Other common answers mention that the application should not be too complicated (cluster size =23%) or time costing (12%). Interestingly, a small subset of the responses (4%) mention safety and privacy of the data."

In [15]:
# count the sum of topic with "melding" in the topic representation
topic_info = topic_model_unwanted_features.get_topic_info()
# topics we are interested in
word_patterns = [r"melding|geluid|trill|piep", r"stress|druk|onrust", r"ingewikkeld|onhandig|moelijk|eenvoudig|complex", r"privacy|veilig|hack|verkop|gegevens", r"fout|vals|verkeerd", r"bovenstaand|alle drie|dat dus",
                 r"tijd kosten"]
topic_info["Representation"] = topic_info["Representation"].astype(str).str.strip("[]").str.replace("'", "").str.replace(", ", "_")
for word_pattern in word_patterns:
    topic_count = topic_info[topic_info['Name'].str.contains(word_pattern, case=False, regex=True)]['Count'].sum()
    # alternatively, count the number of documents containing the word pattern
    docu_count = sum(1 for doc in documents_unwanted_features if pd.Series(doc).str.contains(word_pattern, case=False, regex=True).any())
    print(f"Total count of topics containing '{word_pattern}': {topic_count}, percent: {topic_count / len(documents_unwanted_features) * 100:.2f}%")
    print(f"Total count of documents containing '{word_pattern}': {docu_count}, percent: {docu_count / len(documents_unwanted_features) * 100:.2f}%\n")


Total count of topics containing 'melding|geluid|trill|piep': 375, percent: 43.60%
Total count of documents containing 'melding|geluid|trill|piep': 371, percent: 43.14%

Total count of topics containing 'stress|druk|onrust': 129, percent: 15.00%
Total count of documents containing 'stress|druk|onrust': 179, percent: 20.81%

Total count of topics containing 'ingewikkeld|onhandig|moelijk|eenvoudig|complex': 196, percent: 22.79%
Total count of documents containing 'ingewikkeld|onhandig|moelijk|eenvoudig|complex': 188, percent: 21.86%

Total count of topics containing 'privacy|veilig|hack|verkop|gegevens': 35, percent: 4.07%
Total count of documents containing 'privacy|veilig|hack|verkop|gegevens': 30, percent: 3.49%

Total count of topics containing 'fout|vals|verkeerd': 21, percent: 2.44%
Total count of documents containing 'fout|vals|verkeerd': 19, percent: 2.21%

Total count of topics containing 'bovenstaand|alle drie|dat dus': 9, percent: 1.05%
Total count of documents containing 'bov

## 2. Reasons of flares


In [2]:
# import embeddings and documents
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
 
TEXT_COL = "Reason_flare"
emb_reason_flare = np.load(f"embeddings/emb_robbert_{TEXT_COL}.npy")
documents_reason_flare = pd.read_csv(f"data/answers_for_embedding_{TEXT_COL}.csv")[TEXT_COL].tolist()
print(f"Number of documents: {len(documents_reason_flare)}")
# check length of documents and embeddings
assert len(emb_reason_flare) == len(documents_reason_flare), "Length of embeddings and documents do not match!"
print("Creating BERTopic model for 'Reason_flare'...")
topic_model_reason_flare = BERTopic(verbose=False)

# no UMAP applied
topic_model_reason_flare.umap_model = BaseDimensionalityReduction()
# Agglomerative Clustering with distance threshold
topic_model_reason_flare.hdbscan_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, metric='cosine', linkage='average')
# CountVectorizer with custom stopwords
topic_model_reason_flare.vectorizer_model = CountVectorizer(stop_words=stopwords, min_df=2, ngram_range=(1,2))

# fit and transform
topics_reason_flare, probs_reason_flare = topic_model_reason_flare.fit_transform(documents_reason_flare, embeddings=emb_reason_flare)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 70)
print("Total number of 'Reason_flare' topics: ", len(topic_model_reason_flare.get_topic_info()))
print("Displaying top 20 topic clusters for 'Reason_flare':")
topic_model_reason_flare.get_topic_info().head(20)

Number of documents: 598
Creating BERTopic model for 'Reason_flare'...
Total number of 'Reason_flare' topics:  113
Displaying top 20 topic clusters for 'Reason_flare':


,Topic,Count,Name,Representation,Representative_Docs
0,0,161,0_stress stress_denk stress_denk_factor,"[stress stress, denk stress, denk, factor, stress, drukte, weet stress, langdurige, stress grote, invloed]","[Ik denk stress, denk stress, Ik denk stress]"
1,1,142,1_voeding stress_stress voeding_voeding_eten,"[voeding stress, stress voeding, voeding, eten, stress bepaalde, bepaalde voeding, vet, bepaalde, slecht, slaap]","[Voeding en stress, Bepaalde voeding en stress, Stress en bepaalde voeding maar vooral stress]"
2,2,37,2_opvlamming_veroorzaken_opvlamming stress_opvlamming veroorzaken,"[opvlamming, veroorzaken, opvlamming stress, opvlamming veroorzaken, opvlamming krijg, kwam, krijg, geen opvlamming, veroorzaakt, activiteiten]","[wat ik weet is dat roken, stress, een opvlamming kan veroorzaken. Verder zij er bepaalde dingen die mijn darmen moeilijk verteren zoals bijvoorbeeld mais, erwten, maar ik denk niet dat die ene opvlamming triggeren., de laatste opvlamming kwam door wisseling medicatie. Bij te vet eten ervaar ik verslechtering waarbij ik niet direct een link kan leggen met een opvlamming. Ik ervaar geen stress, geen bijzondere activiteiten waarbij ik denk dat dit een opvlamming veroorzaakt., Stress verergert de opvlamming]"
3,3,30,3_geen idee_idee_geen_helaas,"[geen idee, idee, geen, helaas, , , , , , ]","[Geen idee, Nee, geen idee, Ik heb geen idee.]"
4,4,12,4_helaas weet_weet_helaas_wist,"[helaas weet, weet, helaas, wist, weten, , , , , ]","[Helaas weet ik dat nog niet, Helaas weet ik dat niet., Helaas weet ik dat niet.....]"
5,5,11,5_antibiotica_angstige_infectie_gevoelens,"[antibiotica, angstige, infectie, gevoelens, verkoudheid, buikgriep, griep, griep stress, factor last, covid]","[Stress, antibiotica, Stress, antibiotica, Stress, angstige en sombere gevoelens (dit is echter een vicieuze cirkel en werkt beide kanten uit, de darmproblemen zorgen ook voor een groot deel voor stress, angstige en sombere gevoelens). Grote veranderingen. Verkoudheid / griepvirus.]"
6,6,9,6_buikpijn_diagnose_krijgen_buik,"[buikpijn, diagnose, krijgen, buik, darmen, merk, klachten, krijg, volg, veroorzaakt weet]","[Ik denk dat stress zoals dingen doen die ik liever niet wil doen (zakelijk) de grootste trigger zijn bij mij. Daarnaast is het eten van paddestoelen altijd een reden om buikpijn te krijgen tot heftige krampen aan toe., Stress..ik volg fodmap diert en daardoor geen uijpijn meer en niet meer opgeblazen buik. Tenzij in het eten iets ' mis' gaat dan merk ik dat gelijk in buikpijn en ontlasting. Maar ik noem dat niet gelijk een opvlamming maar heeft wel veel invloed, Ik probeer uit te zoeken of voeding een rol speelt als ik meer buikpijn krijg. Zoals bepaalde soorten groenten, bewerkt vlees, vettige producten, gebruik praktisch geen alcohol meer enz.. Tevens spelen stressige/ zorgelijke periodes een grote rol zoals in maart van dit jaar het overlijden van mijn echtgenote en daaraan voorafgaande de jaren dat ik haar verzorgd heb.]"
7,7,9,7_bepaalde voeding_bepaalde_denk bepaalde_bewerkt voedsel,"[bepaalde voeding, bepaalde, denk bepaalde, bewerkt voedsel, voeding voeding, einde, voeding bepaalde, noten, weet bepaalde, waarvan]","[Voeding: gluten, lactose, bepaalde of combinatie van kruiden. (Waarvan ik nog steeds niet weet welke), Bepaalde voeding, maar niet noodzakelijk. Tijd van het jaar, einde zomer., Bepaalde voeding; granen, suiker en bewerkt voedsel]"
8,8,8,8_werkt_medicatie_duurt_stress medicijnen,"[werkt, medicatie, duurt, stress medicijnen, genomen, medicatie werkt, laatste keer, soms komt, voorspellen, uiteindelijk]","[Voel het wel aankomen, had idee dat medicatie zijn werk niet voldoende deed, klopt uiteindelijk ook, maar had wel eerder wat aan gedaan kunnen worden, duurt ondanks vaste verpleegkundige nog vaak te lang voor serieus genomen te worden, waardoor je nog zieker wordt, Langdurige stress, maar veel laak werkt medicatie niet meer en is het niet te voorspellen qua klachten, Soms stress, soms niks en komt het ge

In [29]:
# print documents in Topic 0 that do not contain the word 
doc_info_reason_flare = topic_model_reason_flare.get_document_info(documents_reason_flare)
print("These are documents in Topic 0 that do not contain the words stress, voed, eten, rust etc just for checking. Some sentences about hormones are included but I guess they are kinda stress related?")
doc_info_reason_flare[(doc_info_reason_flare.Topic == 0) & (~doc_info_reason_flare.Document.str.contains(r"stress|voed|eten|rust", case=False, regex=True)) ]

These are documents in Topic 0 that do not contain the words stress, voed, eten, rust etc just for checking. Some sentences about hormones are included but I guess they are kinda stress related?


,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
37,Teveel werken / overprikkeling / te druk maken,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
94,Zwangerschap = hormonen,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
321,Ingrijpende veranderingen Te weinig vocht drinken Slecht slapen Te veel noten,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
340,"oververmoeidheid, extreem warme dagen",0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
342,Dat blijft koffiedik kijken. Over de jaren lijkt er correlatie met activiteit van het immuunsysteem tijden bv een virale infectie,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
370,Vooral langdurige gestrest zijn.,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
414,Te druk zijn en over mijn grenzen gaan,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False
440,Hormonen; sinds de hormoontherapie tegen borstkanker (minder/geen oestrogeen) voor het eerst (in 35 jaar ziekte) geen ontstekingen in mijn dikke darm.,0,0_stress_voeding_voeding stress_eten,"[stress, voeding, voeding stress, eten, denk stress, denk, stress voeding, factor, bepaalde, trigger]","[Voeding en stress, Voeding en stress, Bepaalde voeding en stress]",stress - voeding - voeding stress - eten - denk stress - denk - stress voeding - factor - bepaalde - trigger,False


For reasons of flare, I think maybe it's sufficient to just report the biggest cluster which includes overlapping themes around stress and voeding? It will be hard to separate these two, because many mention both. About 77%, similar to the results in strict string-searching.

In [7]:
topic_info = topic_model_reason_flare.get_topic_info()
# topics we are interested in
word_patterns = [r"stress|rust|moe|druk|vork|hooi|angst", r"voed|eten|vlees|gluten", r"medicatie|medicijn|antibiotic", r"operatie"]
topic_info["Representation"] = topic_info["Representation"].astype(str).str.strip("[]").str.replace("'", "").str.replace(", ", "_")
for word_pattern in word_patterns:
    topic_count = topic_info[topic_info['Representation'].str.contains(word_pattern, case=False, regex=True)]['Count'].sum()
    # alternatively, count the number of documents containing the word pattern
    docu_count = sum(1 for doc in documents_reason_flare if pd.Series(doc).str.contains(word_pattern, case=False, regex=True).any())
    print(f"Total count of topics containing '{word_pattern}': {topic_count}, percent: {topic_count / len(documents_reason_flare) * 100:.2f}%")
    print(f"Total count of documents containing '{word_pattern}': {docu_count}, percent: {docu_count / len(documents_reason_flare) * 100:.2f}%\n")

Total count of topics containing 'stress|rust|moe|druk|vork|hooi|angst': 434, percent: 72.58%
Total count of documents containing 'stress|rust|moe|druk|vork|hooi|angst': 423, percent: 70.74%

Total count of topics containing 'voed|eten|vlees|gluten': 212, percent: 35.45%
Total count of documents containing 'voed|eten|vlees|gluten': 199, percent: 33.28%

Total count of topics containing 'medicatie|medicijn|antibiotic': 37, percent: 6.19%
Total count of documents containing 'medicatie|medicijn|antibiotic': 41, percent: 6.86%

Total count of topics containing 'operatie': 5, percent: 0.84%
Total count of documents containing 'operatie': 4, percent: 0.67%



## 3. Flare predictability

#### "Yes" answers: 

In [ ]:
# import embeddings and documents
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
 
TEXT_COL = "Wish_predictability_because"
emb_flare_pred = np.load(f"embeddings/emb_robbert_{TEXT_COL}.npy")
df_flare_pred = pd.read_csv(f"data/answers_for_embedding_{TEXT_COL}.csv")
# strip spaces in the column names
df_flare_pred.columns = df_flare_pred.columns.str.strip()
print("Examples of the 'Ja' answers:")
display(df_flare_pred.head())
documents_flare_pred = df_flare_pred[TEXT_COL].tolist()
meta_data = df_flare_pred[["Wish_predictability_flare"]]
# print the value counts of meta_data
print(meta_data["Wish_predictability_flare"].value_counts())
# get the indices of "Ja", "Nee" answers
idxs_ja = meta_data[meta_data["Wish_predictability_flare"] == "Ja"].index.tolist()
idxs_nee = meta_data[meta_data["Wish_predictability_flare"] == "Nee"].index.tolist()
# separate the embeddings and documents based on the indices
emb_flare_pred_jas = emb_flare_pred[idxs_ja]
docs_jas = [documents_flare_pred[i] for i in idxs_ja]
assert len(emb_flare_pred_jas) == len(docs_jas), "Length of 'Ja' embeddings and documents do not match!"
emb_flare_pred_nees = emb_flare_pred[idxs_nee]
docs_nees = [documents_flare_pred[i] for i in idxs_nee]
assert len(emb_flare_pred_nees) == len(docs_nees), "Length of 'Nee' embeddings and documents do not match!"

print("Creating BERTopic model for 'Ja' answers in 'Wish_predictability_because'...")
topic_model_pred_flare_jas = BERTopic(verbose=False)

# no UMAP applied
topic_model_pred_flare_jas.umap_model = BaseDimensionalityReduction()
# Agglomerative Clustering with distance threshold
topic_model_pred_flare_jas.hdbscan_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.65, metric='cosine', linkage='average')
# CountVectorizer with custom stopwords
stopwords.append("opvlamming")
topic_model_pred_flare_jas.vectorizer_model = CountVectorizer(stop_words=stopwords, min_df=2, ngram_range=(1,2))
# fit and transform
topics_reason_flare_jas, probs_reason_flare_jas = topic_model_pred_flare_jas.fit_transform(docs_jas, embeddings=emb_flare_pred_jas)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
print("Number of 'Ja' topics: ", len(topic_model_pred_flare_jas.get_topic_info()))
print("Displaying top 20 topic clusters for 'Ja':")
topic_model_pred_flare_jas.get_topic_info().head(20)
# skil reporting the percentage, use examples 

Examples of the 'Ja' answers:


,ID,Wish_predictability_flare,Wish_predictability_because
0,1,Ja,minder klachten vooraf
1,3,Ja,Dan kan ik mij AANMELDEN VOOR GERICHTE BEHANDELING
2,6,Ja,dan kan ik eerder aan de bel trekken bij MDL verpleegkundige of arts en zelf al dingen in mijn leefstijl proberen aan te passen.
3,7,Ja,Kan eerder ingegrepen worden
4,8,Misschien,Dan zou de arts misschien nog iets kunnen onderdrukken.


Wish_predictability_flare
Ja           518
Misschien    190
Nee           84
Name: count, dtype: int64
Creating BERTopic model for 'Ja' answers in 'Wish_predictability_because'...
Number of 'Ja' topics:  35
Displaying top 20 topic clusters for 'Ja':


,Topic,Count,Name,Representation,Representative_Docs
0,0,161,0_starten_medicijnen_sneller_extra,"[starten, medicijnen, sneller, extra, behandeling, controle, krijgen, contact, ingrijpen, aanpassen]","[Dan kan ik eerder medicijnen starten en wordt erger voorkomen, Dan kan je je medicijnen wellicht al verhogen of andere medicatie starten, Dan kan ik op tijd met medicijnen starten]"
1,1,102,1_anticiperen_mee_rekening mee_mee houden,"[anticiperen, mee, rekening mee, mee houden, rekening, houden, inspelen, actie, maatregelen, ingrijpen]","[Kan ik rekening mee houden, hier rekening mee houden, Dan kan je er rekening mee houden.]"
2,2,55,2_plannen_voorbereiden_agenda_activiteiten,"[plannen, voorbereiden, agenda, activiteiten, erop, houden, rekening, aanpassen, dingen, aanpassen beter]","[Kan je erop voor bereiden en je rekening houden met je activiteiten, Beter mijn activiteiten in plannen, Dan kan ik me erop voorbereiden. Qua werk ook rekening mee houden en op tijd hulp zoeken]"
3,3,53,3_voorkomen_erger_erger voorkomen_klachten,"[voorkomen, erger, erger voorkomen, klachten, voorkomen erger, wellicht, proberen, mogelijk, voorkomt, schade]","[Dan kan ik erger voorkomen, Dan jan je erger nog voorkomen, 1 Nog sneller intervenieren om erger te voorkomen. 2 voorkomen dat mijn suiker uit de bocht vliegt]"
4,4,35,4_rustiger_rust_pakken_weet,"[rustiger, rust, pakken, weet, gaan, rust pakken, denk, slecht, ipv, inplannen]","[Kan ik rustiger aan doen, dan kan ik eerder rustiger aan doen, dan weet ik dat ik het rustiger aan moet doen]"
5,5,27,5_voeding_eten_letten_voeding aanpassen,"[voeding, eten, letten, voeding aanpassen, gaan eten, rekening houden, aanpassen, beter plannen, bewust, mn]","[Mogelijk om wat rustiger aan te doen en op mijn voeding te letten, Kan ik mijn stressniveau bewust naar beneden brengen en letten op eten, Dan zou ik eerder op mijn eten letten of stress voorkomen.]"
6,6,12,6_spelen_handelen_alvast_handig,"[spelen, handelen, alvast, handig, voorbereid, ingrijpen geeft, sneller spelen, ingrijpen, aanpassingen, gedrag]","[Beter voorbereid op aanpassingen in gedrag, Eerder ingrijpen geeft ook rust en planmatig handig, sneller op in kunnen spelen.]"
7,7,12,7_leven_levensstijl_werk_leven aanpassen,"[leven, levensstijl, werk, leven aanpassen, levensstijl aanpassen, passen, aanpassen, qua werk, stellen, beter rekening]","[Dan kan je je leven daarop aanpassen, rekening met werk en het leven, keuzes, Dan kan ik mijn sociale (werk) leven er op aanpassen]"
8,8,7,8_lichaam_bijvoorbeeld_stress_rustiger gaan,"[lichaam, bijvoorbeeld, stress, rustiger gaan, signalen, actie komen, factor, fysiek, leren, mogelijke]","[Ik wil triggers vermijden of verminderen, rust nemen en eventueel snel een behandeling starten., Hoe eerder je weet dat bijvoorbeeld bepaalde waardes omhoog gaan hoe eerder je fysiek, energetisch en mentaal in actie kunt komen om te voorkomen dat je darmen weer gaan ontsteken., Dan betekent dat mijn lichaam signalen kan afgeven om het rustiger aan te gaan doen, of anders te eten bijvoorbeeld, ter (mogelijke) voorkoming.]"
9,9,6,9_ingegrepen_behandeld_snel_maatregelen,"[ingegrepen, behandeld, snel, maatregelen, weet, , , , , ]","[Dak kan er snel worden ingegrepen, Dan kan er eerder ?ingegrepen? worden, Kan eerder ingegrepen worden]"


I would say we report that the most frequent themes were 1. to start medication / see a physician sooner, 2. to prevent / prepare for flares, even though the two themes are quite close, again. 

In [13]:
topic_info = topic_model_pred_flare_jas.get_topic_info()
# topics we are interested in
word_patterns = [r"medicatie|medicijn", r"anticip|plan|voorbereid|voorkomen"]
topic_info["Representation"] = topic_info["Representation"].astype(str).str.strip("[]").str.replace("'", "").str.replace(", ", "_")
for word_pattern in word_patterns:
    topic_count = topic_info[topic_info['Representation'].str.contains(word_pattern, case=False, regex=True)]['Count'].sum()
    # alternatively, count the number of documents containing the word pattern
    docu_count = sum(1 for doc in docs_jas if pd.Series(doc).str.contains(word_pattern, case=False, regex=True).any())
    print(f"Total count of topics containing '{word_pattern}': {topic_count}, percent: {topic_count / len(docs_jas) * 100:.2f}%")
    print(f"Total count of documents containing '{word_pattern}': {docu_count}, percent: {docu_count / len(docs_jas) * 100:.2f}%\n")

Total count of topics containing 'medicatie|medicijn': 161, percent: 31.08%
Total count of documents containing 'medicatie|medicijn': 100, percent: 19.31%

Total count of topics containing 'anticip|plan|voorbereid|voorkomen': 295, percent: 56.95%
Total count of documents containing 'anticip|plan|voorbereid|voorkomen': 140, percent: 27.03%



#### "No" answers: 
These are pretty tricky as the clusters are small and make less sense to me. We should discuss how to present them.

In [16]:
# topic modeling the 'Nee' answers
print("Creating BERTopic model for 'Nee' answers in 'Wish_predictability_because'...")
topic_model_pred_flare_nees = BERTopic(verbose=False)
# no UMAP applied
topic_model_pred_flare_nees.umap_model = BaseDimensionalityReduction()
# Agglomerative Clustering with distance threshold
topic_model_pred_flare_nees.hdbscan_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1, metric='cosine', linkage='complete')
# CountVectorizer with custom stopwords
#remove "niet" from stopwords for 'Nee' analysis
if "niet" in stopwords:
    stopwords.remove("niet")
topic_model_pred_flare_nees.vectorizer_model = CountVectorizer(stop_words=stopwords, min_df=2, ngram_range=(1,2))
# fit and transform
topics_reason_flare_nees, probs_reason_flare_nees = topic_model_pred_flare_nees.fit_transform(docs_nees, embeddings=emb_flare_pred_nees)
print(f"Examples of 'Nee' answers: {len(docs_nees)}")
print("Number of 'Nee' topics: ", len(topic_model_pred_flare_nees.get_topic_info()))
topic_model_pred_flare_nees.get_topic_info().head(20)
# import embeddings and documents

Creating BERTopic model for 'Nee' answers in 'Wish_predictability_because'...
Examples of 'Nee' answers: 84
Number of 'Nee' topics:  4


,Topic,Count,Name,Representation,Representative_Docs
0,0,33,0_niet_voorkomen_niet voorkomen_mee,"[niet, voorkomen, niet voorkomen, mee, geen, idee, medicatie, mee bezig, bezig, goed]","[Het is niet te voorkomen, Ik kan het niet voorkomen, Het valt niet te voorkomen]"
1,1,20,1_weet_komt_goed_geen,"[weet, komt, goed, geen, jaren, geen idee, weten, eraan, idee, rustig]","[Ik weet eigenlijk precies wanneer het eraan komt, Ik weet waar het van komt, tegen woordig weet ik het altijd wanneer het komt]"
2,2,17,2_voel_aankomen_voel aankomen_niet,"[voel, aankomen, voel aankomen, niet, goed, echt, denk niet, lang, niet weten, oorzaak]","[Ik voel het precies aan mijn lichaam, ook al is dat nog enkel een subtiele verandering. Ik doe dan al rustiger aan en licht mijn arts in, Ik voel het nu wel aankomen, Ik voel het al aankomen]"
3,3,14,3_ziek_gaat_geen_krijgen,"[ziek, gaat, geen, krijgen, snel, eraan, rustig, denk, medicatie, niet]","[bij mij gaat het meestal niet om ontstekingen maar verklevingen die vernauwingen veroorzaken, Omgeving maalt er niet om als ik ziek word, Ik voel goed dat het mis gaat zou dan snel werkend medicijn klisma thuis moeten en direct starten klisma]"
